# Case study

In [ ]:
from colorama import Fore
import numpy as np
import pandas as pd
import re
import sys

In [ ]:
DATADIR = "../usb/releases/20201018/"
DROPOUTDATADIR = "../usb/releases/20200302/"
CLIENTSFILE = "clients.csv.gz"
DROPOUTFILE = "dropout.csv.gz"
ANSWERID = "answerId_normalized"
ANSWERIDORIGINAL = "answerId_original"
ANSWERTEXT = "answerText"
ANSWERTITLE = "answerTitle"
CLIENT = "client"
CLIENTID = "clientID"
DROPOUT = "dropout"
QUESTIONNUMBER = "questionNumber"
REMOVED = "REMOVED"
COMPLETERCODE = "2"
DROPOUTCODE = "1"

In [ ]:
def read_dropout_ids():
    dropout = pd.read_csv(DROPOUTDATADIR+DROPOUTFILE)
    dropout_ids = list(dropout[dropout[DROPOUT]==DROPOUTCODE][CLIENTID])
    completer_ids = list(dropout[dropout[DROPOUT]==COMPLETERCODE][CLIENTID])
    return(dropout_ids,completer_ids)

In [ ]:
dropout_ids,completer_ids = read_dropout_ids()
len(dropout_ids),len(completer_ids)

## Task 1: compare metadata of dropouts and completers

In [ ]:
GENDER = "geslacht"
AGE = "leeftijd"
EDUCATION = "opleidng"
DAY1 = "dag1"
MAN = "Man"
WOMAN = "Vrouw"
ANSWER = "answer"
COUNT = "count"
YESNO = "YESNO"
YESNOIDS = ["dagritme","dsm2","dsm3","dsm4","dsm5","dsm6","dsm7","dsm8","dsm9","dsm11",
            "medicijn","drugs","eetdrang","insult","delirium","psych","tabak","canna",
            "coca","speed","xtc","ghb","opiat","sleep","gok","behversl","halluci",
            "suicide","wanen","benniet"]
COLUMNS = [{ANSWER:answer, COUNT:1} for answer in [GENDER,EDUCATION,DAY1,AGE]+YESNOIDS]
CONVERSION = { GENDER: {WOMAN:0,MAN:1},
               EDUCATION: {"Basisschool":0,"LBO/MAVO":1,"MBO":2,"HAVO/VWO":3,"HBO":4,"WO":5,"REMOVED":np.nan},
               YESNO: {"Nee":0,"Ja":1}}

In [ ]:
def make_binary_answers_numeric(df, column_names, binary_translation_table_in):
    data_table = []
    processed_clients = []
    binary_translation_table = dict(binary_translation_table_in)
    column_name_ids = {column_names[i]:i for i in range(0,len(column_names))}
    for i in range(0,len(df)):
        row = df.iloc[i]
        answer_id = row[ANSWERID]
        answer_text = cleanup_answer_text(row[ANSWERTEXT])
        client_id = row[CLIENT]
        if not client_id in processed_clients:
            data_table.append(len(column_names)*[np.nan])
            processed_clients.append(client_id)
        elif client_id != processed_clients[-1]:
            print(Fore.RED, f"make_binary_answers_numeric: cannot happen: 1:{client_id} 2:{processed_clients[-1]}")
        if answer_id in column_names and not pd.isna(answer_text) and answer_text != "removed":
            if not answer_id in binary_translation_table:
                if answer_text == "nee" or answer_text == "ja":
                    binary_translation_table[answer_id] = { "nee": 0, "ja": 1 }
                elif answer_text == "vrouw" or answer_text == "man":
                    binary_translation_table[answer_id] = { "vrouw": 0, "man": 1 }
                else:
                    binary_translation_table[answer_id] = { answer_text: 0 }
            elif answer_text not in binary_translation_table[answer_id]:
                if len(binary_translation_table[answer_id]) == 1:
                    binary_translation_table[answer_id][answer_text] = 1
                else:
                    print(Fore.RED, f"make_binary_answers_numeric: cannot happen: 1:{answer_id} 2:{answer_text} 3:{binary_translation_table[answer_id]}")
            data_table[-1][column_name_ids[answer_id]] = binary_translation_table[answer_id][answer_text]
    return(data_table, binary_translation_table)

In [ ]:
def read_data(dropout_ids,completer_ids):
    client_data = pd.read_csv(DATADIR+CLIENTSFILE)
    dropout_data = client_data[client_data[CLIENT].isin(dropout_ids)]
    completer_data = client_data[client_data[CLIENT].isin(completer_ids)]
    return(dropout_data,completer_data)

In [ ]:
dropout_data,completer_data = read_data(dropout_ids,completer_ids)
all_data = pd.concat([dropout_data,completer_data])

In [ ]:
def verify_data_completeness(dropout_data,completer_data,dropout_ids,completer_ids):
    dropout_data_ids = list(dropout_data[CLIENT])
    for client_id in dropout_ids:
        if client_id not in dropout_data_ids:
            print(f"missing dropout client id: {client_id}")

    completer_data_ids = list(completer_data[CLIENT])
    for client_id in completer_ids:
        if client_id not in completer_data_ids:
            print(f"missing completer client id: {client_id}")
            
verify_data_completeness(dropout_data,completer_data,dropout_ids,completer_ids)

In [ ]:
def get_answer_values(dropout_data, completer_data, column_name):
    nbr_of_dropouts = len(dropout_data.groupby("client").groups)
    nbr_of_completers = len(completer_data.groupby("client").groups)
    dropout_values = pd.DataFrame(dropout_data[dropout_data["answerId_normalized"]==column_name],columns=["answerText"]).groupby("answerText").size()
    completer_values = pd.DataFrame(completer_data[completer_data["answerId_normalized"]==column_name],columns=["answerText"]).groupby("answerText").size()
    nbr_of_missing = nbr_of_dropouts+nbr_of_completers
    answer_values = {}
    for key in dropout_values.index:
        if not key in dropout_values:
            dropout_values[key] = 0
        if not key in completer_values:
            completer_values[key] = 0
        answer_values[key] = dropout_values[key]+completer_values[key]
        nbr_of_missing -= dropout_values[key]+completer_values[key]
    answer_values["MISSING"] = nbr_of_missing
    return(answer_values)

### 1.1 Experiment with binary answer classes

In [ ]:
GESLACHT = "geslacht"
GESLACHT0 = "geslacht0"
GESLACHTT0 = "geslachtt0"
NONQUESTIONS = "^(goTo[0-9]|ltgeslacht1|doel)$"
EXCEPTIONANSWERID = "mdoel"

def cleanup_answer_text(text):
    if pd.isna(text): return(text)
    text = str(text).lower()
    text = re.sub(" *leeftijd in jaren$", "",text)
    text = re.sub("^niet gedronken$", "0",text)
    text = re.sub("11 t/m 15", "13",text)
    text = re.sub("16 t/m 20", "18",text)
    text = re.sub("21 t/m 25", "23",text)
    text = re.sub("26 t/m 30", "28",text)
    text = re.sub("^ja, ik heb deze internetbehandeling al eens gevolgd.$","ja",text)
    text = re.sub("\s+"," ",text)
    text = text.strip()
    return(text)

def normalize_answer_id(answer_id, first_answer_id):
    if first_answer_id == GESLACHT: 
        new_answer_id = answer_id
    elif first_answer_id == GESLACHT0:
        if re.search("0h$",answer_id):
            new_answer_id = re.sub("0h$","h",answer_id)
        else:
            new_answer_id = re.sub("0$","",answer_id)
    elif first_answer_id == GESLACHTT0: 
        new_answer_id = re.sub("t0$","",answer_id)
    else: 
        sys.exit(f"unknown first answer id: {first_answer_id}!")
    if re.search("^(goTo[0-9]|ltgeslacht1|doel)$",answer_id):
        return("")
    if (first_answer_id != GESLACHT and new_answer_id == answer_id and 
        not answer_id == EXCEPTIONANSWERID and not re.search(NONQUESTIONS,answer_id)):
        sys.exit(f"first answer id {first_answer_id} did not change {answer_id}!")
    return(new_answer_id)

In [ ]:
def find_binary_answer_keys(dropout_data, completer_data):
    answer_texts = {}
    data = pd.concat([dropout_data,completer_data])
    for i in range(0,len(data)):
        client_id = data.iloc[i][CLIENT]
        answer_text = cleanup_answer_text(data.iloc[i][ANSWERTEXT])
        answer_title = cleanup_answer_text(data.iloc[i][ANSWERTITLE])
        if pd.isna(answer_title): answer_title = ""
        answer_id = str(data.iloc[i][ANSWERID])
        answer_key = answer_id
        if not answer_key in answer_texts.keys(): answer_texts[answer_key] = []
        if not answer_text in answer_texts[answer_key] and not pd.isna(answer_text) and not answer_text == "removed":
            answer_texts[answer_key].append(answer_text)
    binary_answer_keys = []
    for answer_key in answer_texts:
        if len(answer_texts[answer_key]) == 2:
            binary_answer_keys.append(answer_key)
    return(binary_answer_keys)

In [ ]:
def get_useful_binary_answer_keys(dropout_data, completer_data, binary_answer_keys, show_values=False):
    useful_binary_answer_keys = []
    for key in binary_answer_keys:
        answer_id = key
        answer_values = get_answer_values(dropout_data, completer_data, answer_id)
        if show_values:
            print("#####", answer_id)
            for answer_value in answer_values:
                print(answer_values[answer_value], answer_value)
        nbr_of_values = sum(list(answer_values.values()))
        nbr_of_missing_values = sum([answer_values[answer_value] for answer_value in answer_values if answer_value == "MISSING" or answer_value == "REMOVED"])
        if nbr_of_missing_values < 0.5*nbr_of_values:
            useful_binary_answer_keys.append(answer_id)
    return(useful_binary_answer_keys)

In [ ]:
from sklearn.feature_selection import f_classif, f_regression
import numpy as np
from sklearn.impute import SimpleImputer

def remove_nan(X,y):
    X_no_nan = []
    y_no_nan = []
    for i in range(0,len(X)):
        if not np.isnan(X[i][0]):
            X_no_nan.append([X[i][0]])
            y_no_nan.append(y[i])
    return(X_no_nan, y_no_nan)

def run_anova_classify(dropout_table,completer_table):
    X = dropout_table+completer_table
    y = len(dropout_table)*[DROPOUTCODE]+len(completer_table)*[COMPLETERCODE]
    p_values = []
    for i in range(0,len(X[0])):
        try:
            X_no_nan, y_no_nan = remove_nan([[X[j][i]] for j in range(0,len(X))],y)
            F, p_value = f_classif(X_no_nan, y_no_nan)
            p_values.extend(p_value)
        except:
            p_values.append(np.nan)
    return(p_values)

def column_average(table,column_id):
    try:
        return(np.average([row[column_id] for row in table if not np.isnan(row[column_id])]))
    except:
        return(np.nan)

def count_non_nan(table, column_id):
    return(len([row[column_id] for row in table if not np.isnan(row[column_id])]))

def sort_p_values(p_values,column_names,dropout_table,completer_table):
    return(pd.DataFrame({column_names[i]:(p_values[i],
                                          column_average(completer_table, i),
                                          column_average(dropout_table, i),
                                          count_non_nan(completer_table, i),
                                          count_non_nan(dropout_table, i)
                                         ) for i in sorted(range(0,len(p_values)), key=lambda i:p_values[i])},
                        index=["p_value", "average com", "average dro", "non nan com", "non nan dro"]).T)

In [ ]:
binary_answer_keys = find_binary_answer_keys(dropout_data, completer_data)

In [ ]:
useful_binary_answer_keys = get_useful_binary_answer_keys(dropout_data, completer_data, binary_answer_keys, show_values=False)

In [ ]:
dropout_table_binary, binary_translation_table_dropout = make_binary_answers_numeric(dropout_data, binary_answer_keys, {})
completer_table_binary, binary_translation_table_completer = make_binary_answers_numeric(completer_data, binary_answer_keys, binary_translation_table_dropout)

In [ ]:
binary_translation_table_completer

In [ ]:
p_values = run_anova_classify(dropout_table_binary, completer_table_binary)
sort_p_values(p_values, binary_answer_keys, dropout_table_binary, completer_table_binary)

### 1.2 Experiment with numeric answer classes

In [ ]:
def find_numeric_answer_keys(dropout_data, completer_data):
    numeric_data_seen = {}
    other_data_seen = {}
    data = pd.concat([dropout_data, completer_data])
    for i in range(0,len(data)):
        client_id = data.iloc[i][CLIENT]
        answer_text = cleanup_answer_text(data.iloc[i][ANSWERTEXT])
        answer_title = cleanup_answer_text(data.iloc[i][ANSWERTITLE])
        if pd.isna(answer_title): answer_title = ""
        answer_id = str(data.iloc[i][ANSWERID])
        answer_key = answer_id # +"#"+answer_title
        if not answer_key in numeric_data_seen.keys(): 
            numeric_data_seen[answer_key] = False
            other_data_seen[answer_key] = False
        if not pd.isna(answer_text) and not answer_text == "removed":
            if type(answer_text) == np.float64 or re.search("^[0-9]+$",answer_text): 
                numeric_data_seen[answer_key] = True
            else:
                other_data_seen[answer_key] = True
    return([answer_key for answer_key in numeric_data_seen if numeric_data_seen[answer_key] and not other_data_seen[answer_key]])

In [ ]:
def make_numeric_answers_numeric(df, column_names):
    data_table = []
    processed_clients = []
    column_name_ids = {column_names[i]:i for i in range(0,len(column_names))}
    for i in range(0,len(df)):
        row = df.iloc[i]
        answer_id = row[ANSWERID]
        answer_text = cleanup_answer_text(row[ANSWERTEXT])
        client_id = row[CLIENT]
        if not client_id in processed_clients:
            data_table.append(len(column_names)*[np.nan])
            processed_clients.append(client_id)
        elif client_id != processed_clients[-1]:
            print(Fore.RED, f"make_binary_answers_numeric: cannot happen: 1:{client_id} 2:{processed_clients[-1]}")
        if answer_id in column_names and not pd.isna(answer_text) and answer_text != "removed":
            data_table[-1][column_name_ids[answer_id]] = float(answer_text)
    return(data_table)

In [ ]:
def run_anova_regression(dropout_table, completer_table):
    X = dropout_table+completer_table
    y = len(dropout_table)*[int(DROPOUTCODE)]+len(completer_table)*[int(COMPLETERCODE)]
    p_values = []
    for i in range(0, len(X[0])):
        try:
            X_no_nan, y_no_nan = remove_nan([[X[j][i]] for j in range(0, len(X))],y)
            F, p_value = f_regression(X_no_nan, y_no_nan)
            p_values.extend(p_value)
        except:
            p_values.append(np.nan)
    return(p_values)

In [ ]:
numeric_answer_keys = find_numeric_answer_keys(dropout_data, completer_data)

In [ ]:
completer_table_numeric = make_numeric_answers_numeric(completer_data, numeric_answer_keys)
dropout_table_numeric = make_numeric_answers_numeric(dropout_data, numeric_answer_keys)

In [ ]:
len(numeric_answer_keys), len(dropout_table_numeric[0]), len(completer_table_numeric[0])

In [ ]:
p_values = run_anova_regression(dropout_table_numeric, completer_table_numeric)
sort_p_values(p_values, numeric_answer_keys, dropout_table_numeric, completer_table_numeric)

### 1.3 Experiment with text answer classes

In [ ]:
MINANSWERCOUNT = 10
MAXPVALUE = 0.01
ANSWER = "answer"
COUNT = "count"

def get_binary_answers(data):
    answers_per_client = {}
    for i in range(0,len(data)):
        client_id = data.iloc[i][CLIENT]
        if not client_id in answers_per_client:
            answers_per_client[client_id] = []
        answer_id = str(data.iloc[i][ANSWERID])
        if re.search(NONQUESTIONS,answer_id): continue
        answer_text = cleanup_answer_text(data.iloc[i][ANSWERTEXT])
        answer_title = cleanup_answer_text(data.iloc[i][ANSWERTITLE])
        if not pd.isna(answer_text): # and answer_text != "removed":
            if pd.isna(answer_title): answer_title = ""
            answers_per_client[client_id].append(answer_id+"#"+answer_title+"#"+answer_text)
    return(answers_per_client)

def count_answers(binary_answers):
    answer_counts = {}
    for client_id in binary_answers:
        for answer in binary_answers[client_id]:
            if answer in answer_counts:
                answer_counts[answer] += 1
            else:
                answer_counts[answer] = 1
    return({answer:answer_counts[answer] for answer in sorted(answer_counts.keys(),key=lambda a:answer_counts[a],reverse=True)})

def make_binary_table(data,binary_answers):
    answer_counts = count_answers(binary_answers)
    binary_table = []
    answers_used = []
    for client_id in data[CLIENT].unique():
        binary_table.append([])
        for answer in answer_counts:
            if answer_counts[answer] >= MINANSWERCOUNT:
                if answer in binary_answers[client_id]:
                    binary_table[-1].append(1)
                else:
                    binary_table[-1].append(0)
    for answer in answer_counts:
        if answer_counts[answer] >= MINANSWERCOUNT:
            answers_used.append({ANSWER:answer, COUNT:answer_counts[answer]})
    return(binary_table, answers_used)

def convert_data_to_binary(dropout_data,completer_data):
    all_data = pd.concat([dropout_data,completer_data])
    binary_answers = get_binary_answers(all_data)
    dropout_table_binary, answers_used = make_binary_table(dropout_data,binary_answers)
    completer_table_binary, answers_used = make_binary_table(completer_data,binary_answers)
    return(dropout_table_binary, completer_table_binary, answers_used, binary_answers)

def select_p_values(p_values,column_names,dropout_table,completer_table):
    return({column_names[i][ANSWER]:(p_values[i],column_average(completer_table,i),column_average(dropout_table,i)) 
            for i in sorted(range(0,len(p_values)),key=lambda i:p_values[i])
            if column_average(completer_table,i) < column_average(dropout_table,i) and p_values[i] < MAXPVALUE})

In [ ]:
dropout_table_binary, completer_table_binary, answers_used, binary_answers = convert_data_to_binary(dropout_data,completer_data)

In [ ]:
len(dropout_table_binary[0]),len(completer_table_binary[0]),len(answers_used), len(binary_answers)

In [ ]:
p_values = run_anova(dropout_table_binary,completer_table_binary)

In [ ]:
list(sort_p_values(p_values,answers_used,dropout_table_binary,completer_table_binary).items())[:10]

In [ ]:
def print_answer_ids_freqs(dropout_data,completer_data):
    all_data = pd.concat([dropout_data,completer_data])
    for answer_combi in ["opiat##nee","opiath##0","mateicn10#10.had je gebrek aan onderdak of had je problemen met huisvesting?#niet / geen",
                         "national##nederlands","gokken##nee",
                         "mateicn13#13.had je er moeite mee voor een veilige slaapplaats of voor beschermende kleding te zorgen?#niet / geen"]:
        answer = answer_combi.split("#")[0]
        print(answer,
              len(all_data[all_data[ANSWERIDORIGINAL]==answer]),
              len(all_data[all_data[ANSWERIDORIGINAL]==answer+"0"]),
              len(all_data[all_data[ANSWERIDORIGINAL]==answer+"t0"]))
        
print_answer_ids_freqs(dropout_data,completer_data)

In [ ]:
dropout_predictors = list(select_p_values(p_values,answers_used,dropout_table_binary,completer_table_binary).keys())
print(len(dropout_predictors))

In [ ]:
def get_dropout_predictor_scores(data,dropout_predictors,binary_answers,questionnaire_types):
    scores = []
    for client_id in data[CLIENT].unique():
        score = 0
        for predictor in dropout_predictors:
            if predictor in binary_answers[client_id]: 
                score += 1
        scores.append((score,client_id,questionnaire_types[client_id]))
    return(scores)

def get_questionnaire_types(all_data):
    questionnaire_types = {}
    for i,row in all_data.iterrows():
        client_id = row[CLIENT]
        if not client_id in questionnaire_types: questionnaire_types[client_id] = 0
        questionnaire_types[client_id] += 1
    return(questionnaire_types)

In [ ]:
questionnaire_types = get_questionnaire_types(all_data)
dropout_predictor_scores_dropout = get_dropout_predictor_scores(dropout_data,dropout_predictors,binary_answers,questionnaire_types)
dropout_predictor_scores_completer = get_dropout_predictor_scores(completer_data,dropout_predictors,binary_answers,questionnaire_types)
print([score_tuple[0] for score_tuple in sorted(dropout_predictor_scores_dropout,key=lambda s:s[0],reverse=True)])
print([score_tuple[0] for score_tuple in sorted(dropout_predictor_scores_completer,key=lambda s:s[0],reverse=True)])

In [ ]:
print([score_tuple for score_tuple in sorted(dropout_predictor_scores_dropout,key=lambda s:s[0],reverse=True)][:104])
print([score_tuple for score_tuple in sorted(dropout_predictor_scores_completer,key=lambda s:s[0],reverse=True)][:5])

In [ ]:
print(f"selected 104: 3 completed: {(104-3)/104}")
print(f"selected 245: 32 completed: {(245-32)/245}")

In [ ]:
groups = pd.DataFrame.from_dict(questionnaire_types,orient="index").groupby(0).groups
{g:len(groups[g]) for g in groups}

## Visualization

In [ ]:
import numpy as np
from sklearn.manifold import TSNE

In [ ]:
def print_shape(table):
    print(len(table),len(table[0]))

In [ ]:
def select_columns(dropout_table,completer_table,p_values):
    column_average_dropout = []
    column_average_completer = []
    for i in range(0,len(p_values)):
        column_average_dropout.append(column_average(dropout_table,i))
        column_average_completer.append(column_average(completer_table,i))
    table_in = dropout_table+completer_table
    table_out = []
    for row_in in table_in:
        row_out = [row_in[i] for i in range(0,len(row_in)) 
                   if p_values[i] < MAXPVALUE and
                   column_average_completer[i] < column_average_dropout[i]]
        table_out.append(row_out)
    return(table_out)

In [ ]:
table_dropout_selectors = select_columns(dropout_table_binary,completer_table_binary,p_values)
table_dropout_selectors_dropout = table_dropout_selectors[:len(dropout_table_binary)]
table_dropout_selectors_completer = table_dropout_selectors[len(dropout_table_binary):]

In [ ]:
all_data_array = np.array(dropout_table_binary+completer_table_binary)
all_data_coordinates = TSNE(n_components=2).fit_transform(all_data_array)
dropout_coordinates = all_data_coordinates[:len(dropout_table_binary)]
completer_coordinates = all_data_coordinates[len(dropout_table_binary):]

In [ ]:
print_shape(all_data_coordinates)
print_shape(completer_coordinates)
print_shape(dropout_coordinates)

In [ ]:
import matplotlib.pyplot as plt

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,4))
x = [completer_coordinates[i][0] for i in range(0,len(completer_coordinates))]
y = [completer_coordinates[i][1] for i in range(0,len(completer_coordinates))]
ax1.scatter(x,y,s=10,label="completer")
x = [dropout_coordinates[i][0] for i in range(0,len(dropout_coordinates))]
y = [dropout_coordinates[i][1] for i in range(0,len(dropout_coordinates))]
ax1.scatter(x,y,s=10,label="dropout")
ax1.legend(framealpha=0.5)

ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])
questionnaire_types_list = list(questionnaire_types.values())
for questionnaire_type in set(questionnaire_types.values()):
    x = [all_data_coordinates[i][0] for i in range(0,len(all_data_coordinates)) if questionnaire_types_list[i] == questionnaire_type] 
    y = [all_data_coordinates[i][1] for i in range(0,len(all_data_coordinates)) if questionnaire_types_list[i] == questionnaire_type]
    ax2.scatter(x,y,s=10,label=questionnaire_type)
ax2.legend(framealpha=0.5)
plt.show()